In [8]:
%pip install spark-nlp==6.0.2 numpy pandas matplotlib seaborn setuptools wordcloud pypdf2 contractions unidecode
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init('./spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping_id, window, explode, to_json, from_json
from pyspark.sql.functions import udf, lit, current_timestamp, current_date, date_format
from pyspark.ml import PipelineModel, Pipeline
import PyPDF2
import os

In [10]:

spark = SparkSession.builder \
    .appName("nlpdemo") \
    .config("spark.jars.packages", 
            ",".join([
                "com.johnsnowlabs.nlp:spark-nlp-silicon_2.12:6.0.2"
            ])) \
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

25/07/27 18:29:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer

print(sparknlp.version())
spark = sparknlp.start(apple_silicon=True, memory="8G")

6.0.2


25/07/27 18:29:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:

# Crear un conjunto de textos para analizar
texts = [
    ("Apple Inc. is looking to buy a startup in the United States.",),
    ("Barack Obama was the 44th President of the United States.",),
    ("Elon Musk founded SpaceX, an aerospace manufacturer and space transportation company.",),
    ("The Amazon rainforest is the largest tropical rainforest in the world.",),
    ("Google was founded by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University.",)
]

# Crear un DataFrame
df_ner = spark.createDataFrame(texts, ["texto"])

Modelo utilizado: https://sparknlp.org/api/python/reference/autosummary/sparknlp/annotator/ner/ner_dl/index.html 

Modelo Embeddings: https://sparknlp.org/2020/01/22/glove_100d.html

In [17]:
from sparknlp.annotator import Tokenizer, NerDLModel, WordEmbeddings, NerConverter, WordEmbeddingsModel

document_assembler = DocumentAssembler().setInputCol("texto").setOutputCol("document")

sentence_detector = sparknlp.annotator.SentenceDetector()\
    .setInputCols(["document"]).setOutputCol("sentence")
    
tokenizer = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

# Use WordEmbeddings with glove_100d to match ner_dl requirements
embeddings = WordEmbeddingsModel.pretrained("glove_100d", "en")\
    .setInputCols(["sentence", "token"]).setOutputCol("embeddings")
    
ner_model = NerDLModel.pretrained("ner_dl", "en")\
    .setInputCols(["sentence", "token", "embeddings"]).setOutputCol("ner")
    
ner_converter = NerConverter()\
    .setInputCols(["document", "token", "ner"]).setOutputCol("entidades")

pipeline_ner = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, embeddings, ner_model, ner_converter])
resultado_ner = pipeline_ner.fit(df_ner).transform(df_ner)
resultado_ner.select("texto", explode("entidades").alias("entidad"))\
             .select("entidad.result", "entidad.metadata").show(truncate=False)


glove_100d download started this may take some time.


25/07/27 18:34:44 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 145.3 MB
[ | ]

25/07/27 18:34:44 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/07/27 18:34:44 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ \ ]Download done! Loading the resource.
[OK!]
ner_dl download started this may take some time.


25/07/27 18:34:57 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 13.6 MB
[OK!]


+-------------------+------------------------------------------+
|result             |metadata                                  |
+-------------------+------------------------------------------+
|Apple Inc          |{entity -> ORG, sentence -> 0, chunk -> 0}|
|United States      |{entity -> LOC, sentence -> 0, chunk -> 1}|
|Barack Obama       |{entity -> PER, sentence -> 0, chunk -> 0}|
|United States      |{entity -> LOC, sentence -> 0, chunk -> 1}|
|Elon Musk          |{entity -> ORG, sentence -> 0, chunk -> 0}|
|SpaceX             |{entity -> ORG, sentence -> 0, chunk -> 1}|
|Amazon             |{entity -> LOC, sentence -> 0, chunk -> 0}|
|Google             |{entity -> ORG, sentence -> 0, chunk -> 0}|
|Larry Page         |{entity -> PER, sentence -> 0, chunk -> 1}|
|Sergey Brin        |{entity -> PER, sentence -> 0, chunk -> 2}|
|Ph.D               |{entity -> ORG, sentence -> 0, chunk -> 3}|
|Stanford University|{entity -> ORG, sentence -> 0, chunk -> 4}|
+-------------------+----